In [85]:
"""
ver=2018.05.29. (part 4 미완성)

중앙대학교대학원 경영학과
2018년 1학기 자본시장이론 python 연습
작성자 : 이인호 (박사과정(재무))

아래 실습은 총 4 파트로 구성되어 있음
Part1 : 한국 거래소에서 종목코드를 가져오는 작업
Part2 : 네이버 금융에서 주가 자료를 가져오는 작업
Part3 : 그래프 그리기
Part4 : 매매 시뮬레이션 (미완성)
        (1) [t-5, t-1]에 수익률이 좋은 주식을 매수
        (2) [t, t+5]에 홀딩
        (3) t+6에 매도
        - (1)~(3)의 과정을 반복하며 주식을 매매하는 것의 수익률은 어떨지 확인
"""

In [1]:
"""
Part I .한국거래소(krx)에서 종목코드 가져오기
- KRX 홈페이지의 상장법인목록의 엑셀 파일을 불러오는 작업

0. pandas를 먼저 import함 (사용할 함수)
1. pandas의 read_html은 html 주소에 접속해서 데이터를 가지고 오는 pandas의 명령어로,
   불러온 목록을 code_df에 할당함
   http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13
   위 주소에 접속해보면, 바로 '상장법인목록.xls' 파일이 다운로드됨
2. code_df의 상위 5개 목록만 보여줌
"""

import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.head() # head는 상위 5개만 보여줌. (n)에서 n을 변경할 수 있음. 유사명령어: .tail()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
2,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
4,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시


In [2]:
"""
1. 종목코드는 앞에 0이 와도 6자리여야 하기 때문에 format으로 6자리를 맞춰줌.
2. 위에서 알 수 있듯이 code_df에는 회사명, 종목코드, 업종 등이 있지만,
   code_df에서 종목코드 컬럼의 상위 5개만 보여줌 
"""
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df.종목코드.head()

0    155660
1    078930
2    001250
3    294870
4    011070
Name: 종목코드, dtype: object

In [3]:
"""
1. 전체 개수를 확인해봄
"""
len(code_df)

2367

In [4]:
"""
1. 우리가 필요한 것은 code_df에서 회사명과 종목코드, 둘 뿐이기 때문에
   둘만 남기고 code_df에 다시 할당함
2. 상위 5개만 확인, 업종, 주요제품 등의 컬럼이 모두 삭제되었음
"""
code_df = code_df[['회사명', '종목코드']]
code_df.head()

,회사명,종목코드
0,DSR,155660
1,GS,078930
2,GS글로벌,001250
3,HDC현대산업개발,294870
4,LG이노텍,011070


In [5]:
"""
1. rename 명령어를 통해 한글로된 컬럼명을 영어로 바꿔준 것을 code_df에 할당함
2. code_df에서 공백(" ")을 제거("")해줌
3. 상위 5개만 확인
"""
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
code_df = code_df.replace(" ", "")
code_df.head()

#참고:code_df.to_csv("itemlist.csv") # csv 파일로 저장하는 명령어

,name,code
0,DSR,155660
1,GS,078930
2,GS글로벌,001250
3,HDC현대산업개발,294870
4,LG이노텍,011070


In [30]:
"""
Part II. 네이버금융에서 종목 별 일자 데이터 가져오기
- 네이버에서 제공하는 주가 데이터를 크롤링(crawling)하는 작업
0. 주가를 제공하는 url을 따오는(get) 함수를 사용자가 get_url이라는 이름으로 정의함
   실행을 해도 output은 없음. 함수를 불러오는 것임
"""

def get_url(item_name, code_df):
    """
    <get_url 함수 설명>
    - 먼저, 들어가야 하는 input값으로는 item_name과 code_df가 있음
    ㅇitem_name
     : 종목명은 함수(def) 밖에서 별도로 정의
    ㅇcode_df
     : code_df가 들어가면, 정의한 종목명(item_come)을 key로
       KRX에서 불러온 '종목명, 종목코드' dictionary에서 code를 찾아옴
       code는 아래에서 code_df에서 찾아 정의함
    ㅇurl
     : url의 {code}부분을 임의의 종목코드로 변경하고 인터넷에서 접속하면
       해당 종목의 일별시세페이지가 열림. 즉, url을 정의하는 것은
       위에서 정의된 code를 인터넷 주소(url)의 적당한 위치에 넣어주는 것임.
    ㅇprint
     : 방금 정의한 url을 확인 (필수는 아님)
    ㅇreturn
     : url을 반환(return)해줌으로써 함수 종료
    """
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    code = code.strip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
#     print(type(code))
#     print(len(code))
#     code = code.strip()
#     print(len(code))
    print("요청 URL = {}".format(url))
    return url

In [31]:
"""
1. 특정 기업을 item_name이라는 이름으로 할당함
2. 위에서 정의한 get_url의 일자데이터(daily data) url을 따와서 url에 할당함
   get_url함수 안에 print 명령어가 있으므로,
   해당 종목의 코드가 들어간 url이 출력되는 것을 확인할 수 있음
"""
item_name='삼성전자'
url = get_url(item_name, code_df)

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=005930


In [32]:
"""
1. 일자 데이터를 크롤링해서 담을 DataFrame을 df로 할당함
2. df를 실행해보면 아직은 아무 값도 없는 빈 데이터 시트를 확인할 수 있음
"""
df = pd.DataFrame()
df # df는 아무 값도 없는 empty data sheet임.

""


In [33]:
"""
1. for 반복문을 통해,
   page를 1이상, 21미만(1,21)까지 반복함
   pg_url에서는 url에서 페이지를 넘겨가며 1페이지부터 할당하는 것이고,
   바로 아래 df.append를 통해 해당 html에 있는 값들을 긁어와서 df에 추가(append)해 줌
   이 작업을 1부터 20까지 페이지를 넘겨주며 반복하는 것이 for loop임
2. 상위 5개만 확인해보면 날짜부터 거래량까지 결측치(NaN)도 있지만 가져오긴 한 것을 확인 가능
   1페이지에서 20페이지의 데이터만 가져오기
"""
for page in range(1, 11):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)

df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.08.26,56200.0,200.0,56400.0,56500.0,55700.0,13312711.0
2,2020.08.25,56400.0,300.0,56400.0,56800.0,56100.0,14021705.0
3,2020.08.24,56100.0,200.0,55800.0,56600.0,55400.0,15055896.0
4,2020.08.21,55900.0,500.0,56200.0,56900.0,55800.0,21142288.0


In [34]:
"""
1. 결측치인 NaN을 날려주기 위해(drop) dropna 명령어를 통해 df에 깨끗한 값을 다시 할당
   결측치는 인터넷 페이지 상 표로 표현할 때의 선 등이 공백으로 인식되어 발생한 것 같음
   http://finance.naver.com/item/sise_day.nhn?code=001460&page=1
   위 주소에 접속하여 확인 가능
2. 상위 5개만 확인하면 결측치가 삭제되고 깨끗하게 주가 자료만 남아 있음

"""
df = df.dropna()
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.08.26,56200.0,200.0,56400.0,56500.0,55700.0,13312711.0
2,2020.08.25,56400.0,300.0,56400.0,56800.0,56100.0,14021705.0
3,2020.08.24,56100.0,200.0,55800.0,56600.0,55400.0,15055896.0
4,2020.08.21,55900.0,500.0,56200.0,56900.0,55800.0,21142288.0
5,2020.08.20,55400.0,2400.0,57600.0,57600.0,55300.0,30386029.0


In [35]:
"""
1. df의 크기(행 개수)를 확인. 200개인 것을 알 수 있음.
"""
len(df)

100

In [36]:
"""
1. 종목코드를 추가해주기 위해 code_df에서 특정 종목(name)의 code를 찾아서
   code라는 변수에 할당함
2. 위에서 20페이지를 긁어온 것이 10개씩 20페이지, 즉 200개의 데이터를 맞춰주기 위해
   col이란 변수에 code를 len(df)만큼(200개) 늘려줌
3. 주가 데이터를 크롤링한 df에 col을 code란 컬럼명(column)으로 가장 앞에(loc=0) 붙여줌(insert)
4. 상위 5개만 확인하면 위에서 없었던 code가 맨 앞에 추가된 것 확인 가능
"""
code = code_df[code_df.name == 'BYC'].code.item()
col = [code]*len(df)
df.insert(loc=0, column='code', value=col)
df.head()

,code,날짜,종가,전일비,시가,고가,저가,거래량
1,001460,2020.08.26,56200.0,200.0,56400.0,56500.0,55700.0,13312711.0
2,001460,2020.08.25,56400.0,300.0,56400.0,56800.0,56100.0,14021705.0
3,001460,2020.08.24,56100.0,200.0,55800.0,56600.0,55400.0,15055896.0
4,001460,2020.08.21,55900.0,500.0,56200.0,56900.0,55800.0,21142288.0
5,001460,2020.08.20,55400.0,2400.0,57600.0,57600.0,55300.0,30386029.0


In [37]:
"""
1. 한글로 된 컬럼명을 영어로 바꿔줌(rename 명령어)
"""
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
                         '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
df.head()

,code,date,close,diff,open,high,low,volume
1,001460,2020.08.26,56200.0,200.0,56400.0,56500.0,55700.0,13312711.0
2,001460,2020.08.25,56400.0,300.0,56400.0,56800.0,56100.0,14021705.0
3,001460,2020.08.24,56100.0,200.0,55800.0,56600.0,55400.0,15055896.0
4,001460,2020.08.21,55900.0,500.0,56200.0,56900.0,55800.0,21142288.0
5,001460,2020.08.20,55400.0,2400.0,57600.0,57600.0,55300.0,30386029.0


In [38]:
"""
1. 데이터의 타입을 숫자형(integer)으로 바꿔줌
2. 상위 5개를 확인하면 의미 없는 소수점 1자리의 0이 모두 삭제된 것 확인 가능
"""
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
df.head()

,code,date,close,diff,open,high,low,volume
1,001460,2020.08.26,56200,200,56400,56500,55700,13312711
2,001460,2020.08.25,56400,300,56400,56800,56100,14021705
3,001460,2020.08.24,56100,200,55800,56600,55400,15055896
4,001460,2020.08.21,55900,500,56200,56900,55800,21142288
5,001460,2020.08.20,55400,2400,57600,57600,55300,30386029


In [39]:
"""
1. 컬럼명 date의 타입을 date로 바꿔줌
2. data 컬럼명이 2018.05.29 에서 2018-05-29로 서식이 변경됨
"""
df['date'] = pd.to_datetime(df['date'])
df.head()

,code,date,close,diff,open,high,low,volume
1,001460,2020-08-26,56200,200,56400,56500,55700,13312711
2,001460,2020-08-25,56400,300,56400,56800,56100,14021705
3,001460,2020-08-24,56100,200,55800,56600,55400,15055896
4,001460,2020-08-21,55900,500,56200,56900,55800,21142288
5,001460,2020-08-20,55400,2400,57600,57600,55300,30386029


In [40]:
"""
1. 일자(date)를 기준으로 오름차순 정렬(sort)
2. 가장 앞에 index number는 변경되지 않았지만,
   날짜 순으로 정렬되었고, index number가 함께 따라온 것 확인 가능
"""
df = df.sort_values(by=['date'], ascending=True)
df.head()

,code,date,close,diff,open,high,low,volume
148,001460,2020-04-02,46800,1000,46200,46850,45350,21621076
147,001460,2020-04-03,47000,200,47400,47600,46550,22784682
146,001460,2020-04-06,48700,1700,47500,48800,47250,23395726
145,001460,2020-04-07,49600,900,49650,50200,49000,31524034
144,001460,2020-04-08,48600,1000,49600,49750,48600,25010314


In [42]:
"""
Part 3. 그래프 그리기(plotly)
- plotly라는 그래프 그리는 함수를 사용하여 그래프 그리기
0. plotly 함수를 불러옴
- No module named 'plotly'인 경우 pip install plotly로 모듈 설치
1. 그래프를 그리기 위한 세팅(offline)
2. date를 x축으로, 종가(close)를 y축으로 하는 값들을 trace에 할당
3. data에 trace에 할당된 값을 리스트로 저장
4. 그래프의 레이아웃을 잡음
5. 최종적으로 그려질 그래프를 fig로 할당
6. 그래프 그리기(iplot)
"""
import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode(connected=True)
trace = go.Scatter(x=df.date,y=df.close,name=item_name)
data = [trace]
layout = dict(title='{}의 종가(close) Time Series'.format(item_name),xaxis=dict(rangeselector=dict(buttons=list([dict(count=1,label='1m',step='month',stepmode='backward'),dict(count=3,label='3m',step='month',stepmode='backward'),dict(count=6,label='6m',step='month',stepmode='backward'),dict(step='all')])),rangeslider=dict(),type='date'))
fig = go.Figure(data=data, layout=layout)
offline.iplot(fig)

In [43]:
"""
Part 4. 매매 시뮬레이션 (미완성)
"""
sample = code_df.sample(frac=1).reset_index(drop=True)
sample = sample[:10]
sample

,name,code
0,재영솔루텍,049630
1,엔에이치스팩13호,310840
2,듀오백,073190
3,서호전기,065710
4,네이처셀,007390
5,펄어비스,263750
6,SK디앤디,210980
7,도화엔지니어링,002150
8,영신금속,007530
9,금호타이어,073240


In [44]:
panel_df = pd.DataFrame()

for name in sample['name'] :
    url = get_url(name, sample)

    for page in range(1, 21):
        pg_url = '{url}&page={page}'.format(url=url, page=page)
        panel_df = panel_df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
        panel_df = panel_df.dropna()
   
panel_df = panel_df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
                         '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})

panel_df.reset_index(drop=True, inplace=True)
panel_df.iloc[199:201]

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=049630
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=310840
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=073190
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=065710
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=007390
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=263750
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=210980
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=002150
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=007530
요청 URL = http://finance.naver.com/item/sise_day.nhn?code=073240


,date,close,diff,open,high,low,volume
199,2019.11.07,1075.0,35.0,1110.0,1110.0,1060.0,1262640.0
200,2020.08.26,2060.0,0.0,0.0,0.0,0.0,0.0


In [45]:
code_panel = pd.DataFrame()

for name in sample['name']:
    code = sample[sample.name == name].code.item()
    temp = pd.DataFrame([code]*200)
    code_panel= code_panel.append(temp,ignore_index=True)

code_panel = code_panel.rename(columns= {0: 'code'})
code_panel.iloc[199:201]

,code
199,049630
200,310840


In [46]:
result = pd.concat([code_panel, panel_df], axis=1, join_axes=[code_panel.index])
result.iloc[199:201]

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [47]:
import numpy as np

result['returns'] = np.nan
result.returns = (result.open- result.open.shift(-5)) / result.open #수익률(5일 전 매수, 당일 매도, 시가 기준)
slen = len(sample)*200
for i in range(-1,slen,200):
    result.at[i, 'returns'] = np.nan

result = result.drop(result.index[-1])
result

NameError: name 'result' is not defined

In [108]:
r = []
for i in range(194,slen,200):
    r.append(result.at[i,'returns'])
r.index(max(r))

8

In [109]:
ref = """
"EXCELSIOR"의 블로그를 참고하여 작성함
http://excelsior-cjh.tistory.com/109
"""